# Задание

Обучите и задеплойте модель на основании датасета bigquery-public-data.ml_datasets.census_adult_income (более сложный вариант, требующий преподготовки данных) или bigquery-public-data.ml_datasets.iris. Приведите пример блокнота и скриншоты задеплоенной модели и тестовых предиктов

In [3]:
# pip install xgboost


In [4]:
from google.cloud import storage
from google.cloud import bigquery
from google.oauth2 import service_account

In [5]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder


In [6]:
key_path = "C:\\Users\\peaceful-parity-336514-f361713c806a.json"
print(key_path)

C:\Users\peaceful-parity-336514-f361713c806a.json


In [7]:
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [8]:
print(credentials.project_id)
print(credentials)

peaceful-parity-336514


In [9]:
#Setup the GCP client. 
#Create the object of the client credentials and project_id comes from the key file 
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [10]:
#Query to the bigquery-public-data.ml_datasets.iris
query = """
    SELECT *
    FROM `bigquery-public-data.ml_datasets.iris`
"""

In [11]:
query_job = client.query(query)
df = query_job.to_dataframe()

In [12]:
#print(query_job, "\n")
#print("The query data:")
i = 0
limit = 10
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row)
    i+=1
    if i > limit: break

Row((5.1, 2.5, 3.0, 1.1, 'versicolor'), {'sepal_length': 0, 'sepal_width': 1, 'petal_length': 2, 'petal_width': 3, 'species': 4})
Row((5.0, 2.0, 3.5, 1.0, 'versicolor'), {'sepal_length': 0, 'sepal_width': 1, 'petal_length': 2, 'petal_width': 3, 'species': 4})
Row((5.7, 2.6, 3.5, 1.0, 'versicolor'), {'sepal_length': 0, 'sepal_width': 1, 'petal_length': 2, 'petal_width': 3, 'species': 4})
Row((6.0, 2.2, 4.0, 1.0, 'versicolor'), {'sepal_length': 0, 'sepal_width': 1, 'petal_length': 2, 'petal_width': 3, 'species': 4})
Row((5.8, 2.6, 4.0, 1.2, 'versicolor'), {'sepal_length': 0, 'sepal_width': 1, 'petal_length': 2, 'petal_width': 3, 'species': 4})
Row((5.5, 2.3, 4.0, 1.3, 'versicolor'), {'sepal_length': 0, 'sepal_width': 1, 'petal_length': 2, 'petal_width': 3, 'species': 4})
Row((6.1, 2.8, 4.0, 1.3, 'versicolor'), {'sepal_length': 0, 'sepal_width': 1, 'petal_length': 2, 'petal_width': 3, 'species': 4})
Row((5.5, 2.5, 4.0, 1.3, 'versicolor'), {'sepal_length': 0, 'sepal_width': 1, 'petal_lengt

In [13]:
print(df)

     sepal_length  sepal_width  petal_length  petal_width     species
0             5.1          2.5           3.0          1.1  versicolor
1             5.0          2.0           3.5          1.0  versicolor
2             5.7          2.6           3.5          1.0  versicolor
3             6.0          2.2           4.0          1.0  versicolor
4             5.8          2.6           4.0          1.2  versicolor
..            ...          ...           ...          ...         ...
145           4.7          3.2           1.3          0.2      setosa
146           5.5          3.5           1.3          0.2      setosa
147           4.4          3.0           1.3          0.2      setosa
148           4.4          3.2           1.3          0.2      setosa
149           5.4          3.9           1.3          0.4      setosa

[150 rows x 5 columns]


In [14]:
#Feature definision 
target = df["species"]
features = df.drop("species",1)

C:\Users\emaxkra\AppData\Local\Temp/ipykernel_11156/2892780732.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features = df.drop("species",1)


In [15]:
print("targets:\n",target)
print("==========================================")
print("features:\n",features)

targets:
 0      versicolor
1      versicolor
2      versicolor
3      versicolor
4      versicolor
          ...    
145        setosa
146        setosa
147        setosa
148        setosa
149        setosa
Name: species, Length: 150, dtype: object
features:
      sepal_length  sepal_width  petal_length  petal_width
0             5.1          2.5           3.0          1.1
1             5.0          2.0           3.5          1.0
2             5.7          2.6           3.5          1.0
3             6.0          2.2           4.0          1.0
4             5.8          2.6           4.0          1.2
..            ...          ...           ...          ...
145           4.7          3.2           1.3          0.2
146           5.5          3.5           1.3          0.2
147           4.4          3.0           1.3          0.2
148           4.4          3.2           1.3          0.2
149           5.4          3.9           1.3          0.4

[150 rows x 4 columns]


In [16]:
#Label the target set and encode it into integet 
label_encoder = LabelEncoder()

In [17]:
# just a test of classes output
a = label_encoder.fit(target)
list(a.classes_)

['setosa', 'versicolor', 'virginica']

In [18]:
target_int = label_encoder.fit_transform(target)

In [19]:
print(target_int)

[1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 1 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 2 1 1 2
 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 0 0 0 0 0
 0 0]


In [20]:
#Split data into train and test sets using train_test_split
#sklearn.model_selection.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
X_train, X_test, y_train, y_test = train_test_split(features, target_int, test_size=0.4, random_state=42)

In [21]:
print(X_train)
print(y_train,"length:",len(y_train))

     sepal_length  sepal_width  petal_length  petal_width
123           4.8          3.1           1.6          0.2
24            7.2          3.2           6.0          1.8
25            4.6          3.6           1.0          0.2
23            6.3          3.3           6.0          2.5
94            5.0          3.6           1.4          0.2
..            ...          ...           ...          ...
71            6.3          2.8           5.1          1.5
106           6.3          2.3           4.4          1.3
14            6.0          3.4           4.5          1.6
92            5.1          3.5           1.4          0.2
102           5.5          2.4           3.7          1.0

[90 rows x 4 columns]
[0 2 0 2 0 0 0 2 2 0 2 0 2 0 1 2 0 2 1 0 2 2 0 2 2 0 2 1 1 2 0 0 2 1 1 0 1
 2 0 0 1 2 0 2 2 1 2 0 1 1 2 1 2 2 1 2 1 0 2 0 0 1 0 0 2 0 2 0 2 0 0 1 0 2
 1 2 1 1 0 2 2 2 0 0 2 2 1 1 0 1] length: 90


In [22]:
dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test, label = y_test)

In [40]:
#Train Model 
param = {'max_depth': 3, 'eta': 0.3, 'objective': 'multi:softprob', 'num_class': 3}

#param = {'max_depth': 3, 'eta': 0.3, 'objective': 'survival:cox'}
num_round = 10
bst = xgb.train(param, dtrain, num_round)

[21:22:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [41]:
# Evaluation
pred = bst.predict(dtest)
best_preds = np.asarray([np.argmax(line) for line in pred])
print(precision_score(y_test, best_preds, average='macro'))

0.9298245614035089


In [64]:
# List buckets
storage_client = storage.Client(credentials=credentials, project=credentials.project_id)
buckets = storage_client.list_buckets()
for i in buckets:
    print(i)
    
bucket_info = storage_client.get_bucket("dataproc_lab_bucket")
print(bucket_info._properties)
for i in bucket_info._properties.keys():
    print(i,":",bucket_info._properties[i])

<Bucket: dataproc_lab_bucket>
<Bucket: gcf-sources-981390829592-us-central1>
<Bucket: peaceful-parity-336514.appspot.com>
<Bucket: staging.peaceful-parity-336514.appspot.com>
<Bucket: us.artifacts.peaceful-parity-336514.appspot.com>
{'kind': 'storage#bucket', 'selfLink': 'https://www.googleapis.com/storage/v1/b/dataproc_lab_bucket', 'id': 'dataproc_lab_bucket', 'name': 'dataproc_lab_bucket', 'projectNumber': '981390829592', 'metageneration': '6', 'location': 'US', 'storageClass': 'STANDARD', 'etag': 'CAY=', 'timeCreated': '2022-02-05T05:07:27.989Z', 'updated': '2022-02-06T15:39:26.319Z', 'iamConfiguration': {'bucketPolicyOnly': {'enabled': False}, 'uniformBucketLevelAccess': {'enabled': False}, 'publicAccessPrevention': 'inherited'}, 'locationType': 'multi-region', 'rpo': 'DEFAULT'}
kind : storage#bucket
selfLink : https://www.googleapis.com/storage/v1/b/dataproc_lab_bucket
id : dataproc_lab_bucket
name : dataproc_lab_bucket
projectNumber : 981390829592
metageneration : 6
location : US

In [82]:
# Create bucket ml_model_lab
bucket_name = "ml_model_lab"

#check if the bucket exists
try:
   storage_client.get_bucket(bucket_name)
   print("Bucket:", bucket_name, "exists")
except:
    print("Bucket:", bucket_name, "doesn't exist")
    print("Creating the bucket:", bucket_name)
    bucket = storage_client.bucket(bucket_name)
    bucket.storage_class = "STANDARD"
    bucket.locationType = "multi-region"
    new_bucket = storage_client.create_bucket(bucket, location="us")

Bucket: ml_model_lab doesn't exist
Creating the bucket: ml_model_lab


In [83]:
#Check bucket creation 
bucket_info = storage_client.get_bucket(bucket_name)
print(bucket_info._properties)
for i in bucket_info._properties.keys():
    print(i,":",bucket_info._properties[i])

{'kind': 'storage#bucket', 'selfLink': 'https://www.googleapis.com/storage/v1/b/ml_model_lab', 'id': 'ml_model_lab', 'name': 'ml_model_lab', 'projectNumber': '981390829592', 'metageneration': '1', 'location': 'US', 'storageClass': 'STANDARD', 'etag': 'CAE=', 'timeCreated': '2022-02-12T11:27:30.010Z', 'updated': '2022-02-12T11:27:30.010Z', 'iamConfiguration': {'bucketPolicyOnly': {'enabled': False}, 'uniformBucketLevelAccess': {'enabled': False}, 'publicAccessPrevention': 'inherited'}, 'locationType': 'multi-region', 'rpo': 'DEFAULT'}
kind : storage#bucket
selfLink : https://www.googleapis.com/storage/v1/b/ml_model_lab
id : ml_model_lab
name : ml_model_lab
projectNumber : 981390829592
metageneration : 1
location : US
storageClass : STANDARD
etag : CAE=
timeCreated : 2022-02-12T11:27:30.010Z
updated : 2022-02-12T11:27:30.010Z
iamConfiguration : {'bucketPolicyOnly': {'enabled': False}, 'uniformBucketLevelAccess': {'enabled': False}, 'publicAccessPrevention': 'inherited'}
locationType : mu

In [90]:
# Export Model 
model  = 'model.bst'
bst.save_model(model)
blob = bucket.blob('{}/{}'.format('ml_lab',model))
blob.upload_from_filename(model)